In [6]:
import pandas as pd
import ast # ใช้สำหรับแปลง string ที่หน้าตาเหมือน dictionary/list ให้เป็น object จริงๆ

In [7]:
# --- ฟังก์ชันสำหรับแปลง category ให้อยู่ในรูปแบบ PostgreSQL array ---
def format_category_for_postgres_array(category_str):
    """
    แปลงสตริงที่มี comma-separated values (เช่น "a, b, c")
    ให้อยู่ในรูปแบบ PostgreSQL array literal (เช่น '{"a","b","c"}')
    """
    # ถ้าค่าเป็นค่าว่าง (NaN) หรือสตริงว่าง, ให้คืนค่าเป็น array ว่างของ PostgreSQL
    if pd.isna(category_str) or not category_str:
        return '{}'
    
    # 1. แยกสตริงด้วยเครื่องหมาย comma
    # 2. ตัดช่องว่างที่ไม่จำเป็นออกจากแต่ละส่วน
    # 3. ใส่ double quote (") คร่อมแต่ละส่วน
    items = [f'"{item.strip()}"' for item in category_str.split(',')]
    
    # 4. รวมทุกส่วนเข้าด้วยกันคั่นด้วย comma และครอบทั้งหมดด้วย {}
    return f"{{{','.join(items)}}}"


In [8]:
# --- 1. โหลดไฟล์ CSV ---
try:
    df = pd.read_csv('postPlacesFetch.correct.csv')
    print("File loaded successfully!")
except FileNotFoundError:
    print("Error: 'postPlacesFetch.correct.csv' not found. Please make sure the file is in the same directory as the script.")
    exit()

File loaded successfully!


In [9]:
# --- 2. สร้าง DataFrame ใหม่ตามโครงสร้างของตาราง SQL ---

column_mapping = {
    'api_name': 'name_place',
    'api_formattedAddress': 'address',
    'api_location': 'location',
    'api_rating': 'rating',
    'api_userRatingCount': 'user_rating_total',
    'original_detail': 'sumary_place',
    'api_placeId': 'place_id_by_ggm',
    'api_types': 'category', # คอลัมน์เป้าหมายของเรา
    'api_URI': 'url'
}

df_transformed = df[column_mapping.keys()].rename(columns=column_mapping)

In [10]:
# --- 3. แปลงข้อมูลในแต่ละคอลัมน์ ---

# ฟังก์ชันสำหรับดึงค่า latitude และ longitude
def extract_lat_lon(location_str):
    try:
        location_dict = ast.literal_eval(str(location_str))
        if isinstance(location_dict, dict):
            return location_dict.get('latitude'), location_dict.get('longitude')
    except (ValueError, SyntaxError, TypeError):
        return None, None
    return None, None

df_transformed[['lat', 'lon']] = df_transformed['location'].apply(
    lambda x: pd.Series(extract_lat_lon(x))
)

# จัดการชนิดข้อมูล
df_transformed['rating'] = pd.to_numeric(df_transformed['rating'], errors='coerce')
df_transformed['user_rating_total'] = pd.to_numeric(df_transformed['user_rating_total'], errors='coerce').fillna(0).astype(int)

# เติมค่าว่างในคอลัมน์ที่เป็นข้อความ
text_columns = ['name_place', 'address', 'sumary_place', 'place_id_by_ggm', 'url']
for col in text_columns:
    df_transformed[col] = df_transformed[col].fillna('')
    
# *** ใช้ฟังก์ชันใหม่กับคอลัมน์ category ***
df_transformed['category'] = df_transformed['category'].apply(format_category_for_postgres_array)

In [11]:
# --- 4. จัดเรียงคอลัมน์และสร้างไฟล์ CSV สุดท้าย ---

final_columns = [
    'name_place',
    'address',
    'lat',
    'lon',
    'rating',
    'user_rating_total',
    'sumary_place',
    'place_id_by_ggm',
    'category',
    'url'
]
df_final = df_transformed[final_columns]

output_filename = 'tmp_place_import.csv'
# ใช้ tab เป็นตัวคั่น (delimiter) เพื่อหลีกเลี่ยงปัญหากับ comma ที่อยู่ในข้อมูล
df_final.to_csv(output_filename, index=False, header=True, encoding='utf-8')